In [ ]:
# File: drape.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
# import our frontend
from frontend import App

# make an app
app = App.create()

# create a square mesh resolution 128 spanning the xz plane
V, F = app.mesh.square(res=128, ex=[1, 0, 0], ey=[0, 0, 1])

# add to the asset and name it "sheet"
app.asset.add.tri("sheet", V, F)

# create an icosphere mesh radius 0.5
V, F = app.mesh.icosphere(r=0.5, subdiv_count=4)

# add to the asset and name it "sphere"
app.asset.add.tri("sphere", V, F)

# create a scene "drape-five-sheets"
scene = app.scene.create("drape-five-sheets")

# gap between sheets
gap = 0.01

for i in range(5):

    # add the sheet asset to the scene
    obj = scene.add("sheet")

    # pick two corners
    corner = obj.grab([1, 0, -1]) + obj.grab([-1, 0, -1])

    # place it with an vertical offset and pin the corners
    obj.at(0, gap * i, 0).pin(corner)

    # set fiber directions required for Baraff-Witkin
    obj.direction([1, 0, 0], [0, 0, 1])

# add a sphere mesh at a lower position with jitter and set it static collider
scene.add("sphere").at(0, -0.5 - gap, 0).jitter().pin()

# compile the scene and report stats
fixed = scene.build().report()

# preview the initial scene
fixed.preview()

In [ ]:
# set simulation parameters
param = app.session.param()
param.set("frames", 100)
param.set("dt", 0.01)
param.set("min-newton-steps", 32)

# create a new session with the compiled scene
session = app.session.create(fixed)

# start the simulation and live-preview the results
session.start(param).preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()

In [ ]:
# get a list of log names
logs = session.get.log.names()
assert "time-per-frame" in logs
assert "newton-steps" in logs

# get a list of time per video frame
msec_per_video = session.get.log.numbers("time-per-frame")

# compute the average time per video frame
print("avg per frame:", sum([n for _, n in msec_per_video]) / len(msec_per_video))

# get a list of newton steps
newton_steps = session.get.log.numbers("newton-steps")

# compute the average of consumed newton steps
print("avg newton steps:", sum([n for _, n in newton_steps]) / len(newton_steps))

# Last 8 lines. Omit for everything.
print("==== log stream ====")
for line in session.get.log.stdout(n_lines=8):
    print(line)